In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

import naivebayes as nb

In [2]:
data = pd.read_csv("data.csv")
featureNames = [e for e in data.columns]

# On supprime la premiere colonne
data = data.drop(['filename'],axis=1)

#Test
#data = data.drop(columns=['chroma_stft','rms','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate','spectral_flatness','tonnetz','estimated_tempo','spectral_contrast'])
#data = data.drop(columns=['estimated_tempo'])

# On change les noms des genres par des entiers (de 0 a 9) car notre random forest ne prend en compte que des entiers comme labels

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
labels = encoder.fit_transform(genre_list)
data.iloc[:, -1] = labels

In [3]:
# On separe le dataset en train set et test set (80%/20%)
data2 = data.copy()
train_set = data2.sample(frac=0.80, random_state=random.randint(0,100000))
test_set = data2.drop(train_set.index)

In [4]:
classifier = nb.OurNaiveBayesClassifier()
classifier = classifier.fit(train_set)

new_test_set = test_set.copy()
new_test_set = new_test_set.drop(columns=['genre/label'])

predictions = classifier.predict(new_test_set)
result = classifier.score(predictions, test_set['genre/label'].to_list())

print(result)

#Ne marche pas pour l'instant
#print(classifier.findFeatureImportance(train_set))

0.44
